In [78]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

##**Loading the data**

In [80]:
train=pd.read_csv('/kaggle/input/titanic/train.csv')
test=pd.read_csv('/kaggle/input/titanic/test.csv')
train.head()

In [81]:
train.describe()

In [82]:
##datatype info

In [83]:
train.info()

# Exploratory data Anaylsis****

In [84]:
##categorical attributes
sns.countplot(train['Survived'])

In [85]:
sns.countplot(train['Pclass'])

In [86]:
sns.countplot(train['Sex'])

In [87]:
sns.countplot(train['Embarked'])

In [88]:
sns.countplot(train['Parch'])

In [89]:
sns.countplot(train['SibSp'])

numerical attributes

In [90]:
sns.distplot(train['Age'])

In [91]:
sns.distplot(train['Fare'])

In [92]:
class_fare=train.pivot_table(index='Pclass',values='Fare')
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Avg. Fare')
plt.xticks(rotation=0)

In [93]:
class_fare=train.pivot_table(index='Pclass',values='Fare',aggfunc=np.sum)
class_fare.plot(kind='bar')
plt.xlabel('Pclass')
plt.ylabel('Avg. Fare')
plt.xticks(rotation=0)
plt.title(label='Fare of pclass ')

In [148]:
sns.barplot(data=train,x='Pclass',y='Fare' ,hue='Survived')

In [149]:
sns.barplot(data=train,x='Survived',y='Fare' ,hue='Survived')

**Data Processing**

In [135]:
train_len=len(train)
df=pd.concat([train,test],axis=0)
df=df.reset_index(drop=True)
df.head()

In [136]:
df.tail()

In [137]:
df.isna().sum()

**Drop or delete the column**

In [138]:
df=df.drop(columns=['Cabin'],axis=1)

In [139]:
df.isna().sum()

In [140]:
df['Age'].mean()

In [141]:
#fill missing values using mean of that column
df['Age']=df['Age'].fillna(df['Age'].mean())
df['Fare']=df['Fare'].fillna(df['Fare'].mean())

In [142]:
df['Embarked'].mode()[0]

In [143]:
##fill  missing values using mode of the categorical column
df['Embarked']=df['Embarked'].fillna(df['Embarked'].mode()[0])

# ##log transformation for uniform data distribuiton

In [144]:
sns.distplot(df['Fare'])

In [145]:
df['Fare']=np.log(df['Fare']+1)

In [146]:
sns.distplot(df['Fare'])

# **corelation Matrix ****

In [147]:
corr=df.corr()
plt.figure(figsize=(15,9))
sns.heatmap(corr,annot=True,cmap='coolwarm')

In [150]:
df.head()

**drop unneccessary column**

In [151]:
df=df.drop(columns=['Name','Ticket'],axis=1)
df.head()

# label encoding

In [152]:
from sklearn.preprocessing import LabelEncoder
cols=['Sex','Embarked']
le=LabelEncoder()
for col in cols:
    df[col]=le.fit_transform(df[col])
df.head()

# Train test Split

In [153]:
train=df.iloc[:train_len,:]
test=df.iloc[train_len:,:]

In [154]:
train.head()

In [155]:
test.head()

In [156]:
#input split
X=train.drop(columns=['PassengerId','Survived'],axis=1)
y=train['Survived']

In [157]:
X.head()

# Model Training

In [173]:
#classify column
from sklearn.model_selection import train_test_split,cross_val_score
def classify(model):
    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)
    model.fit(x_train,y_train)
    print('Accuracy:',model.score(x_test,y_test))
    score=cross_val_score(model,X,y,cv=5)
    print(score)
    print('CV Score:',np.mean(score))

In [174]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
classify(model)

In [175]:
from sklearn.tree import DecisionTreeClassifier
model=DecisionTreeClassifier()
classify(model)

In [177]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier()
classify(model)

In [178]:
from sklearn.ensemble import ExtraTreesClassifier
model=ExtraTreesClassifier()
classify(model)

In [180]:
from xgboost import XGBClassifier
model=XGBClassifier()
classify(model)

In [181]:
from lightgbm import LGBMClassifier
model=LGBMClassifier()
classify(model)

In [183]:
from catboost import CatBoostClassifier
model=CatBoostClassifier(verbose=0)
classify(model)

# complete model training with full data

In [184]:
model=LGBMClassifier()
model.fit(X,y)

In [185]:
test.head()

In [191]:
#input split for test data
X_test=test.drop(columns=['PassengerId','Survived'],axis=1)

In [192]:
X_test.head()

In [193]:
pred=model.predict(X_test)
pred

# Test Submission

In [194]:
sb=pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
sb.head()

In [198]:
sb.info()

In [200]:
sb['Survived']=pred
sb['Survived']=sb['Survived'].astype('int')

In [201]:
sb.head()

In [203]:
sb.to_csv('submission.csv',index=False)